# Census Data Imports (Work in Progress)

The US Census is a fantastic source of free demographic data. Thankfully, we can easily access large amounts of this data at once via Python.

Suppose you receive this letter from a graduating NVCU senior:

Let's say that some NVCU graduating seniors are interested in settling down and raising a family a few years after they graduate. Therefore, they'd like to know what parts of the US have especially high percentages of married families with kids. However, they'd also like to avoid having to spend too much money on a home, so they're interested in regions with relatively low property costs. And finally, because they'd prefer to live in a growing region rather than a declining one, they want to know which areas have the highest 5-year population growth rates.

In order to answer these questions, we'll use the Census API to retrieve marriage, home price, and population growth data for all US counties. We'll then combine this data together in order to create a 'Fit Index' that we can share with these seniors.

By Kenneth Burchfiel

Released under the MIT License

In [39]:
import pandas as pd

# Part 1: Introducing the Census API

## Importing a Census API Key

You can obtain a free Census API key [at this website.](https://api.census.gov/data/key_signup.html). The following cell imports my own personal key, so you'll need to replace this code with one that loads in your own API key.

In [27]:
with open ('census_api_key_path.txt') as file:
    key_path = file.read()
with open(key_path) as file:
    key = file.read()

In [40]:
key

'eac3512bdfd84bf4a86f2051d6298bb57d1bbc9d'

Note: There is also a 'census' Python library (available via pypi and conda) that helps simplify the process of requesting API data. You may choose to use it for your own Census research, but I ended up not needing it for the data retrieval tasks shown below. In addition, foregoing the library allowed me to demonstrate how to retrieve data directly from an API, which you may find helpful when working with APIs that don't have a corresponding Python library. 

In [38]:
## Example of reading data from the Census library into a Pandas DataFrame:
from census import Census
c = Census(key)
pd.DataFrame(c.acs5.get(('NAME', 'B01001_001E'),
{'for': 'county:*'}))

,NAME,B01001_001E,state,county
0,"Autauga County, Alabama",58761.0,01,001
1,"Baldwin County, Alabama",233420.0,01,003
2,"Barbour County, Alabama",24877.0,01,005
3,"Bibb County, Alabama",22251.0,01,007
4,"Blount County, Alabama",59077.0,01,009
...,...,...,...,...
3217,"Vega Baja Municipio, Puerto Rico",54182.0,72,145
3218,"Vieques Municipio, Puerto Rico",8199.0,72,147
3219,"Villalba Municipio, Puerto Rico",21984.0,72,149
3220,"Yabucoa Municipio, Puerto Rico",30313.0,72,151


As discussed above, we will bypass the Census Python library and work directly with the Census API. Thankfully, the Census has detailed API documentation that makes this process relatively straightforward.

[This list](https://api.census.gov/data/2022/acs/acs5/examples.html) (for the 2022 American Community Survey) provides a set of example API calls. For instance, here's one of the sample URLs shown on this page for retrieving county-level data:

https://api.census.gov/data/2022/acs/acs5?get=NAME,B01001_001E&for=county:*&key=YOUR_KEY_GOES_HERE

(B01001_001E refers to the total population estimates for a given county. We can find this out by going to the [Detailed Tables](https://api.census.gov/data/2022/acs/acs5/variables.html) page for the 2022 American Community Survey and navigating to the row with a 'Name' value of B01001_001E. This link, which may take a while to load is available on the [API Documentation Page](https://www.census.gov/data/developers/data-sets/acs-5year.html) for the 5-Year-Estimates version of the American Community Survey.

If you replace the 'YOUR_KEY_GOES_HERE' component of the URL with your actual key, then enter this link into your web browser, you'll receive a very long list of counties, population values, and state and county codes. The top of the list looks like this:

In [ ]:
[["NAME","B01001_001E","state","county"],
["Autauga County, Alabama","58761","01","001"],
["Baldwin County, Alabama","233420","01","003"],
["Barbour County, Alabama","24877","01","005"],
["Bibb County, Alabama","22251","01","007"],
["Blount County, Alabama","59077","01","009"],
["Bullock County, Alabama","10328","01","011"],

We can create a similar list within Python by using the requests library to retrieve data from this URL, then convert it to JSON format:

In [64]:
# The following code borrows from the requests library documentation at 
# https://docs.python-requests.org/en/latest/index.html
import requests
r = requests.get(f'https://api.census.gov/data/2022/\
acs/acs5?get=NAME,B01001_001E&for=county:*&key={key}')
# Printing the first 300 characters of this output:
print("r.text:\n",r.text[0:300],'\n')
# Printing the first 5 lines of r.json:
print("r.json:\n",r.json()[0:5],'\n')

r.text:
 [["NAME","B01001_001E","state","county"],
["Autauga County, Alabama","58761","01","001"],
["Baldwin County, Alabama","233420","01","003"],
["Barbour County, Alabama","24877","01","005"],
["Bibb County, Alabama","22251","01","007"],
["Blount County, Alabama","59077","01","009"],
["Bullock County, Ala 

r.json:
 [['NAME', 'B01001_001E', 'state', 'county'], ['Autauga County, Alabama', '58761', '01', '001'], ['Baldwin County, Alabama', '233420', '01', '003'], ['Barbour County, Alabama', '24877', '01', '005'], ['Bibb County, Alabama', '22251', '01', '007']] 



Converting our response to JSON allows it to be easily read into a Pandas DataFrame, as shown below:

In [70]:
# pd.DataFrame(r.text) # Produces the following error:
# "ValueError: DataFrame constructor not properly called!"
pd.DataFrame(r.json()).head()

,0,1,2,3
0,NAME,B01001_001E,state,county
1,"Autauga County, Alabama",58761,01,001
2,"Baldwin County, Alabama",233420,01,003
3,"Barbour County, Alabama",24877,01,005
4,"Bibb County, Alabama",22251,01,007


I wanted to introduce the requests library here because you'll likely find it useful for extracting data from the internet. However, in this case, we can read Census data directly into a Pandas DataFrame via pd.read_json():

In [73]:
df_results = pd.read_json(
    f'https://api.census.gov/data/2022/\
acs/acs5?get=NAME,B01001_001E&for=county:*&key={key}')
# read_json documentation:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html
df_results.head()

,0,1,2,3
0,NAME,B01001_001E,state,county
1,"Autauga County, Alabama",58761,01,001
2,"Baldwin County, Alabama",233420,01,003
3,"Barbour County, Alabama",24877,01,005
4,"Bibb County, Alabama",22251,01,007


In [74]:
# At this point, the DataFrame's columns are [0, 1, 2, 3], whereas the columns
# we want to use are stored within the first row. The following code 
# sets these row values as our column values, then deletes this row:

df_results.columns = df_results.iloc[0]
df_results.drop(0, inplace = True)
df_results

df_results

,NAME,B01001_001E,state,county
1,"Autauga County, Alabama",58761,01,001
2,"Baldwin County, Alabama",233420,01,003
3,"Barbour County, Alabama",24877,01,005
4,"Bibb County, Alabama",22251,01,007
5,"Blount County, Alabama",59077,01,009
...,...,...,...,...
3218,"Vega Baja Municipio, Puerto Rico",54182,72,145
3219,"Vieques Municipio, Puerto Rico",8199,72,147
3220,"Villalba Municipio, Puerto Rico",21984,72,149
3221,"Yabucoa Municipio, Puerto Rico",30313,72,151


# Part 2: Retrieving our Data